# *Mapping-only experiment for test experiments to find LGN*

## Run the update/reset shortcut on the desktop before each experiment
***
***
# **Without mouse on stage**

In [8]:
import contextlib
import time

import np_config
import np_jobs
import np_logging
import np_services
import np_workflows
from np_workflows import npxc
import np_workflows.experiments.task_trained_network as TTN

np_logging.getLogger()

np_workflows.elapsed_time_widget()

***
## Quiet mode
**on**  [*default*]
- error details are hidden
- regular messages displayed (log level = INFO)

**off**
- full error details (with traceback)
- extra messages displayed (log level = DEBUG)

In [9]:
np_workflows.quiet_mode_widget()

ToggleButton(value=True, button_style='info', description='Quiet mode is on', icon='check', tooltip='Quiet mod…

***
## Launch apps via RSC
[optional]

In [ ]:
with contextlib.suppress(Exception):
    np_services.start_rsc_apps()

***
## Select mouse and user

In [ ]:
user, mouse = np_workflows.user_and_mouse_widget()

***
## Select `EPHYS`
Re-run cell this cell if mouse ID is changed

In [ ]:
selected_session = TTN.stim_session_select_widget(mouse)

***
## Generate new session
Check mouse ID and session are correct: this cell will lock them in!

In [ ]:
experiment: TTN.Ephys | TTN.Hab = TTN.new_experiment(mouse, user, selected_session.session)

session, platform_json = experiment.session, experiment.session.platform_json

platform_json.workflow_start_time = npxc.now()
hab: bool = isinstance(experiment, TTN.Hab)

***
## Checks before starting

### **Ephys day 2?** 
Don't forget to adjust probe targeting!

In [ ]:
np_workflows.check_hardware_widget()

In [ ]:
if not hab:
    np_workflows.check_openephys_widget()

***
## Setup, test, reset all components
*This cell must not be skipped!*

In [ ]:
experiment.initialize_and_test_services()

***
## Dip probes

In [ ]:
if not hab:
    print(np_workflows.dye_info_widget.__doc__)
    np_workflows.dye_info_widget(session)

## Photodoc of probes in dye

In [ ]:
if not hab:
    np_workflows.photodoc_widget('pre_experiment_surface_image')

## Probe depths in dye

In [ ]:
if not hab:
    np_workflows.probe_depth_widget(session)

***
***
# **With mouse on stage**
## Before lowering cartridge

In [ ]:
platform_json.HeadFrameEntryTime = npxc.now()
np_workflows.wheel_height_widget(session)
np_workflows.check_mouse_widget()

***
## When cartridge is lowered

### Set zoom to 4.0 for photodocs of brain
- focus on the brain surface

## Photodoc of brain (tap probes if hab Day1 or Day2)

In [ ]:
platform_json.CartridgeLowerTime = npxc.now()
np_workflows.photodoc_widget('brain_surface_image')

***
## ISI map

In [ ]:
np_workflows.isi_widget(mouse.lims)

***
## Probe insertion

In [ ]:
if not hab:
    platform_json.ProbeInsertionStartTime = npxc.now()

***
## Photodoc before advancing probes

In [ ]:
if not hab:
    np_workflows.photodoc_widget('pre_insertion_surface_image')

***
## Settle timer & insertion notes

- run both cells now: settle timer will start

- fill out probe notes while waiting

- press Save once

- notes are saved when the timer finishes (button will turn green to confirm)

In [ ]:
if not hab:
    np_workflows.insertion_notes_widget(session)

In [ ]:
np_workflows.print_countdown_timer(minutes=40)

***
## Photodoc after probes settled, before experiment

In [ ]:
if not hab:
    np_workflows.photodoc_widget('post_insertion_surface_image')

In [ ]:
np_workflows.pre_stim_check_widget()

***
## Start devices recording

In [ ]:
last_exception = Exception()
attempts = 3
while attempts:
    np_logging.getLogger().info('Waiting for recorders to finish processing') 
    while not all(r.is_ready_to_start() for r in experiment.recorders):
        time.sleep(1)
    np_logging.getLogger().info('Recorders ready')     
    try:
        experiment.start_recording()
    except AssertionError as exc:
        np_logging.getLogger().info('`experiment.start_recording` failed: trying again')
        attempts -= 1
        last_exception = exc              # exc only exists within the try block
    
    else:
        break
else:
    np_logging.getLogger().error(f'`experiment.start_recording` failed after multiple attempts', exc_info=last_exception)
    raise last_exception

***
# Make mapping stimulus longer than usual
Gabors only, remove plain flash stims (which aren't used for QC RFs)

In [ ]:
stim = 'mapping'
params = experiment.params
params[stim]['max_total_duration_minutes'] = 21
params[stim]['default_gabor_duration_seconds'] = 20 * 60
params[stim]['default_flash_duration_seconds'] = 10

***
## Start stimulus script
| mapping |

In [ ]:

np_services.ScriptCamstim.params = params[stim]
np_services.ScriptCamstim.script = experiment.scripts[stim]

np_services.ScriptCamstim.start()

with contextlib.suppress(Exception):
    np_logging.web(f'lgn_test_{experiment.ttn_session.name.lower()}').info(f"{stim.capitalize()} {stim} started")

with contextlib.suppress(Exception):
    while not np_services.ScriptCamstim.is_ready_to_start():
        time.sleep(2.5)
    
if isinstance(np_services.ScriptCamstim, np_services.Finalizable):
    np_services.ScriptCamstim.finalize()

with contextlib.suppress(Exception):
    np_logging.web(f'lgn_test_{experiment.ttn_session.name.lower()}').info(f"{stim.capitalize()} {stim} finished")

***
## Stop recording

In [ ]:
experiment.stop_recording_after_stim_finished()

***
## Before removing probes

In [ ]:
if not hab:
    np_workflows.photodoc_widget('post_stimulus_surface_image')

***
## After fully retracting probes

In [ ]:
if not hab:
    np_workflows.photodoc_widget('post_experiment_surface_image')

***
## After raising cartridge

In [ ]:
platform_json.HeadFrameExitTime = npxc.now()

---
## Save notebook and allow workflow to finish while removing mouse

First, save the notebook:
- `Ctrl + S`
- or `File > Save Notebook` 

Apart from `finishing_checks`, the remaining cells require no input.

They will take a while, so set them running now, then deal with the finishing checks:
- `Shift + Enter` until you reach the end 
- or `Run > Run Selected Cell and Below`

In [ ]:
np_workflows.finishing_checks_widget()

## Finalize

In [ ]:
platform_json.workflow_complete_time = npxc.now()

experiment.finalize_services(*experiment.recorders, *experiment.stims)
experiment.validate_services(*experiment.recorders, *experiment.stims)

## Copy data

In [ ]:
experiment.copy_files()

# Add to post-experiment pipeline

**hab**
- add session to QC queue

**ephys**
- add session to np-exp upload queue, specifying this rig's Acq as `hostname`
    - ensures checksum-validated copy of ephys data on np-exp
    - then adds session to spike-sorting queue
    - then adds session to QC queue

In [ ]:
if hab:
    np_jobs.PipelineQCQueue().add_or_update(session, priority=99)
else:
    np_jobs.PipelineNpexpUploadQueue().add_or_update(session, hostname=np_config.Rig().Acq, priority=99)